In [71]:
%load_ext autoreload
%autoreload 2

import sys
import pandas as pd
import matplotlib.pyplot as plt


sys.path.append('../../src')
from utils.formatter import rename_columns, pivot_database, apply_sct_gain, dt_remover
from utils.functions import clean_folder

from di.functions import append_di_column

from paths import DB_DIR, PVTDB, RAW_DB_DIR, PRE_FMTD_DB_DIR, FMTD_DB_DIR

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


#### Clean destine diretory

In [72]:
#clean_folder(PRE_FMTD_DB_DIR_FMTD_DB_DIR, "*.csv")
#clean_folder(FMTD_DB_DIR, "*.csv")

### Download database

In [73]:
#download_raw_db()

In [74]:
#df = pd.read_csv(RAW_DB_DIR+"/mqtt_database.csv")

In [75]:
#df.head(5)

In [87]:
# Load data
db = pivot_database(RAW_DB_DIR, 'mqtt_database.csv')

In [77]:
db.head(2)

DL_AC                                           \
                    R47CH0/Irms R47CH0/PF R47CH0/Pwr R47CH0/Vrms SHTC3/H   
date                                                                       
2021-02-16 12:44:00         NaN       NaN        NaN         NaN     NaN   
2021-02-16 12:46:00         NaN       NaN        NaN         NaN     NaN   

                              DL_CY                                ...  \
                    SHTC3/T DHT11/H DHT11/T R22CH0/Irms R22CH0/PF  ...   
date                                                               ...   
2021-02-16 12:44:00     NaN    95.0    26.4     24.2331    0.8205  ...   
2021-02-16 12:46:00     NaN    95.0    26.4     24.3757    0.8214  ...   

                          DL_MR                                               \
                    R82CH0/Irms R82CH0/PF R82CH0/Pwr R82CH0/Vrms R82CH1/Irms   
date                                                                           
2021-02-16 12:44:00         NaN       NaN        NaN         NaN         NaN   
2021-02-16 12:46:00         NaN       NaN        NaN         NaN         NaN   

                                                                      
                    R82CH1/PF R82CH1/Pwr R82CH1/Vrms SHTC3/H SHTC3/T  
date                                                                  
2021-02-16 12:44:00       NaN        NaN         NaN     NaN     NaN  
2021-02-16 12:46:00       NaN        NaN         NaN     NaN     NaN  

[2 rows x 37 columns]

In [78]:
# Remove time error inserted by ESP32 timer
db = dt_remover(db)

In [79]:
# Apply current sensors gain to data
apply_sct_gain(db)

In [80]:
# Save changes
db.to_csv(PRE_FMTD_DB_DIR+PVTDB, float_format='%.4f')

In [81]:
# Split data
db_house = db['DL_CY'].drop(['R47CH1/Irms', 'R47CH1/PF', 'R47CH1/Pwr', 'R47CH1/Vrms'], axis=1)
db_pump = db['DL_CY'].drop(['R22CH0/Irms', 'R22CH0/PF', 'R22CH0/Pwr', 'R22CH0/Vrms'], axis=1)
db_air = db['DL_AC'].copy()
db_fridge = db['DL_FG'].copy()
db_fan = db['DL_MR'].drop(['R82CH0/Irms', 'R82CH0/PF', 'R82CH0/Pwr', 'R82CH0/Vrms'], axis=1)
db_computer = db['DL_MR'].drop(['R82CH1/Irms','R82CH1/PF', 'R82CH1/Pwr', 'R82CH1/Vrms'], axis=1)
db_ex = db['DL_EX'].copy()

In [82]:
# Naming datasets
db_house.name = 'house'
db_pump.name = 'pump'
db_air.name = 'air'
db_fridge.name = 'fridge'
db_fan.name = 'fan'
db_computer.name = 'computer'
db_ex.name = 'external'

In [83]:
# Rename columns
rename_columns(db_house)
rename_columns(db_pump)
rename_columns(db_air)
rename_columns(db_fridge)
rename_columns(db_fan)
rename_columns(db_computer)
db_ex.columns = ['H.0', 'T.0', 'LL', 'H', 'T']

In [84]:
# Append IDT
datasets = [db_house, db_pump, db_air, db_fridge, db_fan, db_computer, db_ex]

for dataset in datasets:
    append_di_column(dataset)

In [85]:
# Drop unused data
for dataset in datasets:
    try:
        dataset.drop(['Vrms','Irms','PF'], axis=1, inplace=True)
    except Exception as e:
        print(e)

"['Vrms' 'Irms' 'PF'] not found in axis"


In [86]:
# Export data
for dataset in datasets:
    dataset.to_csv(FMTD_DB_DIR+"/{}.csv".format(dataset.name), float_format='%.4f')